In [10]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [11]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [12]:
%%writefile -a reducer.py

print "%s\t%d" % ("test_test", 500)

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)
    


Appending to reducer.py


In [5]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2018-08-01 17:15 /data/wiki/en_articles_part


In [13]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

test_test	500
test_test	500
0%however	1
0)).(1	2
0,03	1
0,1,0	1
0-03-071979-8	1
0-06-063447-2	1
0-08-025211-7	1
0-10-551856-5	1


rm: `wordcount_result_1539038202617901': No such file or directory
18/10/08 22:36:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/08 22:36:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/08 22:36:46 INFO mapred.FileInputFormat: Total input files to process : 1
18/10/08 22:36:46 INFO mapreduce.JobSubmitter: number of splits:2
18/10/08 22:36:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1539012427123_0003
18/10/08 22:36:47 INFO impl.YarnClientImpl: Submitted application application_1539012427123_0003
18/10/08 22:36:47 INFO mapreduce.Job: The url to track the job: http://56f4304f331c:8088/proxy/application_1539012427123_0003/
18/10/08 22:36:47 INFO mapreduce.Job: Running job: job_1539012427123_0003
18/10/08 22:36:53 INFO mapreduce.Job: Job job_1539012427123_0003 running in uber mode : false
18/10/08 22:36:53 INFO mapreduce.Job:  map 0% reduce 0%
18/10/08 22:37:09 INFO mapreduce.Job:  map 29% reduce 0%
18/10/08 22:3